<a href="https://colab.research.google.com/github/sherry975/CSC-791-Project/blob/master/embedding_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# !ls "/content/drive/My Drive/"

import os
os.chdir("/content/drive/My Drive/CSC791-NLP/791 NLP project/791_project_repo")

import matplotlib.pyplot as plt

import pickle

import sys
import traceback

import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm

import nltk
nltk.download('punkt')
nltk.download('wordnet')

import datetime

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate


def print_time(*args):
    text = ""
    for arg in args:
        text += arg
    print("[",datetime.datetime.now(),"]", text)


def pickle_save(filename, data2pkl):
    fileObject = open(filename,'wb') 
    pickle.dump(data2pkl,fileObject)
    fileObject.close()

        
def pickle_load(filename):
    fileObject = open(filename,'rb') 
    data_unpkl = pickle.load(fileObject)
    fileObject.close()
    return data_unpkl

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# get embeddings from trained SDAE
# https://github.com/fh295/SentenceRepresentation/pull/5

import theano
import theano.tensor as tensor
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import pickle as pkl
import numpy
import copy
import os
import pdb
from scipy import optimize, stats
from collections import OrderedDict
# from sklearn.cross_validation import KFold

from nltk.tokenize import wordpunct_tokenize

profile = False

# push parameters to Theano shared variables
def zipp(params, tparams):
    for kk, vv in params.items():
        tparams[kk].set_value(vv)


# pull parameters from Theano shared variables
def unzip(zipped):
    new_params = OrderedDict()
    for kk, vv in zipped.items():
        new_params[kk] = vv.get_value()
    return new_params


# get the list of parameters: Note that tparams must be OrderedDict
def itemlist(tparams):
    return [vv for kk, vv in tparams.items()]


# dropout
def dropout_layer(state_before, use_noise, trng):
    proj = tensor.switch(use_noise,
                         state_before * trng.binomial(state_before.shape, p=0.5, n=1, dtype=state_before.dtype),
                         state_before * 0.5)
    return proj


# make prefix-appended name
def _p(pp, name):
    return '%s_%s' % (pp, name)


# initialize Theano shared variables according to the initial parameters
def init_tparams(params):
    tparams = OrderedDict()
    for kk, pp in params.items():
        tparams[kk] = theano.shared(params[kk], name=kk)
    return tparams


# load parameters
def load_params(path, params):
    pp = numpy.load(path)
    # print("pp: ")
    # print(pp)
    # for p in pp:
    #     print(p)
    for kk, vv in params.items():
        if kk not in pp:
            raise Warning('%s is not in the archive' % kk)
        params[kk] = pp[kk]
    # print("end pp.")
    return params


# layers: 'name': ('parameter initializer', 'feedforward')
layers = {'ff': ('param_init_fflayer', 'fflayer'),
          'gru': ('param_init_gru', 'gru_layer'),
          'gru_cond': ('param_init_gru_cond', 'gru_cond_layer'),
          }


def get_layer(name):
    fns = layers[name]
    return (eval(fns[0]), eval(fns[1]))


# some utilities
def ortho_weight(ndim):
    W = numpy.random.randn(ndim, ndim)
    u, s, v = numpy.linalg.svd(W)
    return u.astype('float32')


def norm_weight(nin, nout=None, scale=0.01, orth=True):
    if nout == None:
        nout = nin
    if nout == nin and orth:
        W = ortho_weight(nin)
    else:
        W = scale * numpy.random.randn(nin, nout)
    return W.astype('float32')


def tanh(x):
    return tensor.tanh(x)


def rectifier(x):
    return tensor.maximum(0., x)


def linear(x):
    return x


# feedforward layer: affine transformation + point-wise nonlinearity
def param_init_fflayer(options, params, prefix='ff', nin=None, nout=None, orth=True):
    if nin == None:
        nin = options['dim_proj']
    if nout == None:
        nout = options['dim_proj']
    params[_p(prefix, 'W')] = norm_weight(nin, nout, scale=0.01, orth=orth)
    params[_p(prefix, 'b')] = numpy.zeros((nout,)).astype('float32')

    return params


def fflayer(tparams, state_below, options, prefix='rconv', activ='lambda x: tensor.tanh(x)', **kwargs):
    return eval(activ)(tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')])


# GRU layer
def param_init_gru(options, params, prefix='gru', nin=None, dim=None, hiero=False):
    if nin == None:
        nin = options['dim_proj']
    if dim == None:
        dim = options['dim_proj']
    if not hiero:
        W = numpy.concatenate([norm_weight(nin, dim),
                               norm_weight(nin, dim)], axis=1)
        params[_p(prefix, 'W')] = W
        params[_p(prefix, 'b')] = numpy.zeros((2 * dim,)).astype('float32')
    U = numpy.concatenate([ortho_weight(dim),
                           ortho_weight(dim)], axis=1)
    params[_p(prefix, 'U')] = U

    Wx = norm_weight(nin, dim)
    params[_p(prefix, 'Wx')] = Wx
    Ux = ortho_weight(dim)
    params[_p(prefix, 'Ux')] = Ux
    params[_p(prefix, 'bx')] = numpy.zeros((dim,)).astype('float32')

    return params


def gru_layer(tparams, state_below, options, prefix='gru', mask=None, **kwargs):
    nsteps = state_below.shape[0]
    if state_below.ndim == 3:
        n_samples = state_below.shape[1]
    else:
        n_samples = 1

    dim = tparams[_p(prefix, 'Ux')].shape[1]

    if mask == None:
        mask = tensor.alloc(1., state_below.shape[0], 1)

    def _slice(_x, n, dim):
        if _x.ndim == 3:
            return _x[:, :, n * dim:(n + 1) * dim]
        return _x[:, n * dim:(n + 1) * dim]

    state_below_ = tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')]
    state_belowx = tensor.dot(state_below, tparams[_p(prefix, 'Wx')]) + tparams[_p(prefix, 'bx')]
    U = tparams[_p(prefix, 'U')]
    Ux = tparams[_p(prefix, 'Ux')]

    def _step_slice(m_, x_, xx_, h_, U, Ux):
        preact = tensor.dot(h_, U)
        preact += x_

        r = tensor.nnet.sigmoid(_slice(preact, 0, dim))
        u = tensor.nnet.sigmoid(_slice(preact, 1, dim))

        preactx = tensor.dot(h_, Ux)
        preactx = preactx * r
        preactx = preactx + xx_

        h = tensor.tanh(preactx)

        h = u * h_ + (1. - u) * h
        h = m_[:, None] * h + (1. - m_)[:, None] * h_

        return h  # , r, u, preact, preactx

    seqs = [mask, state_below_, state_belowx]
    _step = _step_slice

    rval, updates = theano.scan(_step,
                                sequences=seqs,
                                outputs_info=[tensor.alloc(0., n_samples, dim)],
                                # None, None, None, None],
                                non_sequences=[tparams[_p(prefix, 'U')],
                                               tparams[_p(prefix, 'Ux')]],
                                name=_p(prefix, '_layers'),
                                n_steps=nsteps,
                                profile=profile,
                                strict=True)
    rval = [rval]
    return rval


# Conditional GRU layer without Attention
def param_init_gru_cond(options, params, prefix='gru_cond', nin=None, dim=None, dimctx=None):
    if nin == None:
        nin = options['dim']
    if dim == None:
        dim = options['dim']
    if dimctx == None:
        dimctx = options['dim']

    params = param_init_gru(options, params, prefix, nin=nin, dim=dim)

    # context to LSTM
    Wc = norm_weight(dimctx, dim * 2)
    params[_p(prefix, 'Wc')] = Wc

    Wcx = norm_weight(dimctx, dim)
    params[_p(prefix, 'Wcx')] = Wcx

    return params


def gru_cond_layer(tparams, state_below, options, prefix='gru',
                   mask=None, context=None, one_step=False,
                   init_memory=None, init_state=None,
                   context_mask=None,
                   **kwargs):
    assert context, 'Context must be provided'

    if one_step:
        assert init_state, 'previous state must be provided'

    nsteps = state_below.shape[0]
    if state_below.ndim == 3:
        n_samples = state_below.shape[1]
    else:
        n_samples = 1

    # mask
    if mask == None:
        mask = tensor.alloc(1., state_below.shape[0], 1)

    dim = tparams[_p(prefix, 'Ux')].shape[1]

    # initial/previous state
    if init_state == None:
        init_state = tensor.alloc(0., n_samples, dim)

    # projected context
    assert context.ndim == 2, 'Context must be 2-d: #sample x dim'
    pctx_ = tensor.dot(context, tparams[_p(prefix, 'Wc')])
    pctxx_ = tensor.dot(context, tparams[_p(prefix, 'Wcx')])

    def _slice(_x, n, dim):
        if _x.ndim == 3:
            return _x[:, :, n * dim:(n + 1) * dim]
        return _x[:, n * dim:(n + 1) * dim]

    # projected x
    state_belowx = tensor.dot(state_below, tparams[_p(prefix, 'Wx')]) + tparams[_p(prefix, 'bx')]
    state_below_ = tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')]

    def _step_slice(m_, x_, xx_, h_, pctx_, pctxx_, U, Ux):
        preact = tensor.dot(h_, U)
        preact += x_
        preact += pctx_
        preact = tensor.nnet.sigmoid(preact)

        r = _slice(preact, 0, dim)
        u = _slice(preact, 1, dim)

        preactx = tensor.dot(h_, Ux)
        preactx *= r
        preactx += xx_
        preactx += pctxx_

        h = tensor.tanh(preactx)

        h = u * h_ + (1. - u) * h
        h = m_[:, None] * h + (1. - m_)[:, None] * h_

        return h

    seqs = [mask, state_below_, state_belowx]
    _step = _step_slice

    shared_vars = [tparams[_p(prefix, 'U')],
                   tparams[_p(prefix, 'Ux')]]

    if one_step:
        rval = _step(*(seqs + [init_state, pctx_, pctxx_] + shared_vars))
    else:
        rval, updates = theano.scan(_step,
                                    sequences=seqs,
                                    outputs_info=[init_state],
                                    non_sequences=[pctx_,
                                                   pctxx_] + shared_vars,
                                    name=_p(prefix, '_layers'),
                                    n_steps=nsteps,
                                    profile=profile,
                                    strict=True)
    return rval


# initialize all parameters
def init_params(options):
    params = OrderedDict()
    # embedding: shared
    params['Wemb'] = norm_weight(options['n_words'], options['dim_word'])
    # encoder
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder',
                                              nin=options['dim_word'],
                                              dim=options['dim'])
    init_state = get_layer('ff')[0](options, params, prefix='ff_state',
                                    nin=options['dim'], nout=options['dim'])
    # decoder
    params = get_layer(options['decoder'])[0](options, params, prefix='decoder',
                                              nin=options['dim_word'],
                                              dim=options['dim'],
                                              dimctx=options['dim'])
    # readout
    params = get_layer('ff')[0](options, params, prefix='ff_logit',
                                nin=options['dim'], nout=options['n_words'])

    return params

# build a training model
def build_model(tparams, options):
    trng = RandomStreams(1234)
    use_noise = theano.shared(numpy.float32(0.))

    # clean string: #words x #samples
    x = tensor.matrix('x', dtype='int64')
    x_mask = tensor.matrix('x_mask', dtype='float32')
    # noisy string: #words x #samples
    if options['use_preemb']:
        x_noise = tensor.tensor3('x_noise', dtype='float32')
    else:
        x_noise = tensor.matrix('x_noise', dtype='int64')
    xn_mask = tensor.matrix('x_noise_mask', dtype='float32')

    n_timesteps = x_noise.shape[0]
    n_samples = x_noise.shape[1]

    # word embedding (source)
    if options['use_preemb']:
        emb = x_noise
    else:
        emb = tparams['Wemb'][x_noise.flatten()].reshape([n_timesteps, n_samples, options['dim_word']])
    # encoder
    proj = get_layer(options['encoder'])[1](tparams, emb, options,
                                            prefix='encoder',
                                            mask=xn_mask)
    ctx = proj[0][-1]  # the last hidden state is the context
    init_state = get_layer('ff')[1](tparams, ctx, options,
                                    prefix='ff_state', activ='tanh')

    n_timesteps = x.shape[0]
    n_samples = x.shape[1]

    # word embedding (target)
    emb = tparams['Wemb'][x.flatten()].reshape([n_timesteps, n_samples, options['dim_word']])
    emb_shifted = tensor.zeros_like(emb)
    emb_shifted = tensor.set_subtensor(emb_shifted[1:], emb[:-1])
    emb = emb_shifted
    # decoder
    proj = get_layer(options['decoder'])[1](tparams, emb, options,
                                            prefix='decoder',
                                            mask=x_mask, context=ctx,
                                            one_step=False,
                                            init_state=init_state)
    proj_h = proj
    # compute word probabilities
    logit = get_layer('ff')[1](tparams, proj_h, options, prefix='ff_logit', activ='linear')
    logit_shp = logit.shape
    probs = tensor.nnet.softmax(logit.reshape([logit_shp[0] * logit_shp[1], logit_shp[2]]))
    # cost
    x_flat = x.flatten()
    x_flat_idx = tensor.arange(x_flat.shape[0]) * options['n_words'] + x_flat
    cost = -tensor.log(probs.flatten()[x_flat_idx] + 1e-8)
    cost = cost.reshape([x.shape[0], x.shape[1]])
    cost = (cost * x_mask).sum(0)
    cost = cost.mean()

    return trng, use_noise, x, x_mask, x_noise, xn_mask, ctx, cost


# build a sampler
def build_sampler(tparams, options, trng):
    if options['use_preemb']:
        x = tensor.tensor3('x', dtype='float32')
    else:
        x = tensor.matrix('x', dtype='int64')
    xr = x[::-1]
    n_timesteps = x.shape[0]
    n_samples = x.shape[1]

    # word embedding (source)
    if options['use_preemb']:
        emb = x
    else:
        emb = tparams['Wemb'][x.flatten()].reshape([n_timesteps, n_samples, options['dim_word']])
    # encoder
    proj = get_layer(options['encoder'])[1](tparams, emb, options, prefix='encoder')

    ctx = proj[0][-1]
    ctx_mean = ctx

    init_state = get_layer('ff')[1](tparams, ctx_mean, options, prefix='ff_state', activ='tanh')

    print('Building f_init...',
          outs=[init_state, ctx])
    f_init = theano.function([x], outs, name='f_init', profile=profile)
    print('Done')

    # x: 1 x 1
    y = tensor.vector('y_sampler', dtype='int64')
    init_state = tensor.matrix('init_state', dtype='float32')
    ctx = tensor.matrix('ctx', dtype='float32')

    # if it's the first word, emb should be all zero
    emb = tensor.switch(y[:, None] < 0,
                        tensor.alloc(0., 1, tparams['Wemb'].shape[1]),
                        tparams['Wemb'][y])
    proj = get_layer(options['decoder'])[1](tparams, emb, options,
                                            prefix='decoder',
                                            mask=None, context=ctx,
                                            one_step=True,
                                            init_state=init_state)
    next_state = proj

    logit = get_layer('ff')[1](tparams, proj, options, prefix='ff_logit', activ='linear')
    logit_shp = logit.shape
    next_probs = tensor.nnet.softmax(logit)
    next_sample = trng.multinomial(pvals=next_probs).argmax(1)

    # next word probability
    print('Building f_next..', )
    inps = [y, ctx, init_state]
    outs = [next_probs, next_sample, next_state]
    f_next = theano.function(inps, outs, name='f_next', profile=profile)
    print('Done')

    return f_init, f_next


# generate sample
def gen_sample(tparams, f_init, f_next, x, options, trng=None, k=1, maxlen=30,
               stochastic=True, argmax=False):
    if k > 1:
        assert not stochastic, 'Beam search does not support stochastic sampling'

    sample = []
    sample_score = []
    if stochastic:
        sample_score = 0

    live_k = 1
    dead_k = 0

    hyp_samples = [[]] * live_k
    hyp_scores = numpy.zeros(live_k).astype('float32')
    hyp_states = []

    ret = f_init(x)
    next_state, ctx0 = ret[0], ret[1]
    next_w = -1 * numpy.ones((1,)).astype('int64')

    for ii in range(maxlen):
        ctx = numpy.tile(ctx0, [live_k, 1])
        inps = [next_w, ctx, next_state]
        ret = f_next(*inps)
        next_p, next_w, next_state = ret[0], ret[1], ret[2]

        if stochastic:
            if argmax:
                nw = next_p[0].argmax()
            else:
                nw = next_w[0]
            sample.append(nw)
            sample_score += next_p[0, nw]
            if nw == 0:
                break
        else:
            cand_scores = hyp_scores[:, None] - numpy.log(next_p)
            cand_flat = cand_scores.flatten()
            ranks_flat = cand_flat.argsort()[:(k - dead_k)]

            voc_size = next_p.shape[1]
            trans_indices = ranks_flat / voc_size
            word_indices = ranks_flat % voc_size
            costs = cand_flat[ranks_flat]

            new_hyp_samples = []
            new_hyp_scores = numpy.zeros(k - dead_k).astype('float32')
            new_hyp_states = []

            for idx, [ti, wi] in enumerate(zip(trans_indices, word_indices)):
                new_hyp_samples.append(hyp_samples[ti] + [wi])
                new_hyp_scores[idx] = copy.copy(costs[ti])
                new_hyp_states.append(copy.copy(next_state[ti]))

            # check the finished samples
            new_live_k = 0
            hyp_samples = []
            hyp_scores = []
            hyp_states = []

            for idx in range(len(new_hyp_samples)):
                if new_hyp_samples[idx][-1] == 0:
                    sample.append(new_hyp_samples[idx])
                    sample_score.append(new_hyp_scores[idx])
                    dead_k += 1
                else:
                    new_live_k += 1
                    hyp_samples.append(new_hyp_samples[idx])
                    hyp_scores.append(new_hyp_scores[idx])
                    hyp_states.append(new_hyp_states[idx])
            hyp_scores = numpy.array(hyp_scores)
            live_k = new_live_k

            if new_live_k < 1:
                break
            if dead_k >= k:
                break

            next_w = numpy.array([w[-1] for w in hyp_samples])
            next_state = numpy.array(hyp_states)

    if not stochastic:
        # dump every remaining one
        if live_k > 0:
            for idx in range(live_k):
                sample.append(hyp_samples[idx])
                sample_score.append(hyp_scores[idx])

    return sample, sample_score


def recon_err(f_recon_err, prepare_data, data, iterator,
              corrupt=None, verbose=False,
              use_preemb=False, wv_emb=None):
    n_samples = len(data[0])
    probs = numpy.zeros((n_samples, 1)).astype('float32')

    n_done = 0

    for _, valid_index in iterator:
        x, mask = prepare_data([valid[0][t] for t in valid_index])
        if corrupt is not None:
            x_noise = corrupt(x)
        else:
            x_noise = copy.copy(x)
        if use_preemb:
            shp = x_noise.shape
            x_noise = wv_embs[x_noise.flatten()].reshape([shp[0], shp[1], wv_embs.shape[1]])
        pred_probs = f_recon_err(x, x_noise, mask)
        probs[valid_index] = pred_probs[:, None]

        n_done += len(valid_index)
        if verbose:
            print('%d/%d samples computed' % (n_done, n_samples))

    return probs


# optimizers
# name(hyperp, tparams, grads, inputs (list), cost) = f_grad_shared, f_update
def adadelta(lr, tparams, grads, inp, cost):
    zipped_grads = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_grad' % k) for k, p in tparams.items()]
    running_up2 = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_rup2' % k) for k, p in tparams.items()]
    running_grads2 = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_rgrad2' % k) for k, p in
                      tparams.items()]

    zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
    rg2up = [(rg2, 0.95 * rg2 + 0.05 * (g ** 2)) for rg2, g in zip(running_grads2, grads)]

    f_grad_shared = theano.function(inp, cost, updates=zgup + rg2up)

    updir = [-tensor.sqrt(ru2 + 1e-6) / tensor.sqrt(rg2 + 1e-6) * zg for zg, ru2, rg2 in
             zip(zipped_grads, running_up2, running_grads2)]
    ru2up = [(ru2, 0.95 * ru2 + 0.05 * (ud ** 2)) for ru2, ud in zip(running_up2, updir)]
    param_up = [(p, p + ud) for p, ud in zip(itemlist(tparams), updir)]

    f_update = theano.function([lr], [], updates=ru2up + param_up, on_unused_input='ignore')

    return f_grad_shared, f_update


def adam(lr, tparams, grads, inp, cost):
    gshared = [theano.shared(p.get_value() * 0., name='%s_grad' % k) for k, p in tparams.items()]
    gsup = [(gs, g) for gs, g in zip(gshared, grads)]

    f_grad_shared = theano.function(inp, cost, updates=gsup)

    lr0 = 0.0002
    b1 = 0.1
    b2 = 0.001
    e = 1e-8

    updates = []

    i = theano.shared(numpy.float32(0.))
    i_t = i + 1.
    fix1 = 1. - b1 ** (i_t)
    fix2 = 1. - b2 ** (i_t)
    lr_t = lr0 * (tensor.sqrt(fix2) / fix1)

    for p, g in zip(tparams.values(), gshared):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * tensor.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (tensor.sqrt(v_t) + e)
        p_t = p - (lr_t * g_t)
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    updates.append((i, i_t))

    f_update = theano.function([lr], [], updates=updates, on_unused_input='ignore')

    return f_grad_shared, f_update


def rmsprop(lr, tparams, grads, inp, cost):
    zipped_grads = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_grad' % k) for k, p in tparams.items()]
    running_grads = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_rgrad' % k) for k, p in tparams.items()]
    running_grads2 = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_rgrad2' % k) for k, p in
                      tparams.items()]

    zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
    rgup = [(rg, 0.95 * rg + 0.05 * g) for rg, g in zip(running_grads, grads)]
    rg2up = [(rg2, 0.95 * rg2 + 0.05 * (g ** 2)) for rg2, g in zip(running_grads2, grads)]

    f_grad_shared = theano.function(inp, cost, updates=zgup + rgup + rg2up)

    updir = [theano.shared(p.get_value() * numpy.float32(0.), name='%s_updir' % k) for k, p in tparams.items()]
    updir_new = [(ud, 0.9 * ud - 1e-4 * zg / tensor.sqrt(rg2 - rg ** 2 + 1e-4)) for ud, zg, rg, rg2 in
                 zip(updir, zipped_grads, running_grads, running_grads2)]
    param_up = [(p, p + udn[1]) for p, udn in zip(itemlist(tparams), updir_new)]
    f_update = theano.function([lr], [], updates=updir_new + param_up, on_unused_input='ignore')

    return f_grad_shared, f_update


def sgd(lr, tparams, grads, inp, cost):
    gshared = [theano.shared(p.get_value() * 0., name='%s_grad' % k) for k, p in tparams.items()]
    gsup = [(gs, g) for gs, g in zip(gshared, grads)]

    f_grad_shared = theano.function(inp, cost, updates=gsup)

    pup = [(p, p - lr * g) for p, g in zip(itemlist(tparams), gshared)]
    f_update = theano.function([lr], [], updates=pup)

    return f_grad_shared, f_update


def perplexity(f_cost, lines, worddict, options, verbose=False, wv_embs=None):
    n_lines = len(lines)
    cost = 0.
    n_words = 0.

    for i, line in enumerate(lines):
        # get array from line
        wordin = wordpunct_tokenize(line.strip())
        seq = [worddict[w] if w in worddict else 1 for w in wordin]
        seq = [s if s < options['n_words'] else 1 for s in seq]
        n_words += len(seq) + 1
        x = numpy.array(seq + [0]).astype('int64').reshape([len(seq) + 1, 1])
        x_mask = numpy.ones((len(seq) + 1, 1)).astype('float32')
        if options['use_preemb']:
            shp = x.shape
            xi = wv_embs[x.flatten()].reshape([shp[0], shp[1], wv_embs.shape[1]])
        else:
            xi = x
        cost_one = f_cost(x, x_mask, xi, x_mask) * (len(seq) + 1)
        cost += cost_one

        if verbose:
            print('Sentence ', i, '/', n_lines, ' (', seq.mean(), '):', 2 ** (cost_one / len(seq) / numpy.log(2)), ', ',
                  cost_one / len(seq))
    cost = cost / n_words
    return cost


def padded(indices, maxlen):
    if len(indices) > maxlen:
        indices = indices[:maxlen]
    elif len(indices) < maxlen:
        indices = indices + [1] * (maxlen - len(indices))
    return indices + [0]


def get_embs(f_ctx, lines, worddict, options):
    nw = options["n_words"]
    n_ex = len(lines)
    batch_idx = range(int(n_ex / 1000))
    remainder = (-(n_ex % 1000),n_ex)
    indices = [(i*1000,(i+1)*1000) for i in batch_idx]
    indices.append(remainder)
    embs = []
    for i in indices:
        batch = lines[i[0]:i[1]]
        seq = []
        for i, line in enumerate(batch):
            maxlen = options["maxlen"]
            # indices = [worddict[w.decode("utf-8")] \
            #             if w.decode("utf-8") in worddict \
            #             and worddict[w.decode("utf-8")] < nw\
            #             else 1 for w in line]

            indices = [worddict[w] \
                        if w in worddict \
                        and worddict[w] < nw\
                        and  w != "#"\
                        else 1 for w in line[7]]
            # print("padded(indices, maxlen)", padded(indices, maxlen))
            seq.append(padded(indices, maxlen))
        seq = numpy.array(seq).astype('int64')
        x = numpy.swapaxes(seq,0,1)
        x_mask = numpy.ones((seq.shape[1], seq.shape[0])).astype('float32')
        # print("x_mask: ", x_mask)
        emb = f_ctx(x, x_mask)
        embs.extend(emb)
    return embs


def embedding(
          model='model.npz',
          save='embeddings.txt',
          sentences='sentences.txt',
          dictionary='sentences.txt.dict.pkl',
):
    
    # load dictionary
    with open(dictionary, 'rb') as f:
        worddict = pkl.load(f)
    # reload options
    with open('%s.pkl'%model, 'rb') as f:
        model_options = pkl.load(f)

    print('Loading data')
    examples = []
    # classes = []
    examples = pickle_load(sentences)
    # with open(sentences, 'r') as f:
        # for l in f:
        #     # examples.append(l.split("\t")[0].lower().split())
        #     examples.append(l.lower().split())
        #     # classes.append(l.split("\t")[1].strip())

    print('Building model')
    params = init_params(model_options)
    print("init_params: ", params.keys())

    # reload parameters
    params = load_params(model, params)
    print("loaded params.")

    tparams = init_tparams(params)
    trng, use_noise, \
          x, x_mask, x_noise, xn_mask, \
          ctx, \
          cost = \
          build_model(tparams, model_options)

    # sentence representation
    print('Building f_ctx...')
    f_ctx = theano.function([x_noise, xn_mask], ctx)
    
    print('Getting embeddings...')
    embs = get_embs(f_ctx, examples, worddict, model_options)
    print('Saving...')
    fout = open(save, "w")
    # for e, c in zip(*(embs, classes)):
    #     line = " ".join([str(d) for d in e]) + "\t" + c + "\n"
    #     fout.write(line)
    for e in embs:
        line = " ".join([str(d) for d in e]) + "\n"
        fout.write(line)
    fout.close()
    print("Done.")

# main entrance
if __name__ == '__main__':
    embedding( model='model/marked_model_corrected_1.npz', 
            save='output_data/sdae_emb_corrected_1_train_tmp.txt',
            sentences='output_data/training_masked_list.pkl',
            dictionary='output_data/training_masked_list.pkl.dict.pkl'
            )
    
    print_time("Done.")

Loading data
Building model
init_params:  odict_keys(['Wemb', 'encoder_W', 'encoder_b', 'encoder_U', 'encoder_Wx', 'encoder_Ux', 'encoder_bx', 'ff_state_W', 'ff_state_b', 'decoder_W', 'decoder_b', 'decoder_U', 'decoder_Wx', 'decoder_Ux', 'decoder_bx', 'decoder_Wc', 'decoder_Wcx', 'ff_logit_W', 'ff_logit_b'])
loaded params.
Building f_ctx...


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


KeyboardInterrupt: ignored

In [ ]:
# model = "model/marked_model_corrected_1.npz"
# with open('%s.pkl'%model, 'rb') as f:
#     model_options = pkl.load(f)
# print(model_options)
# pp = numpy.load(model)
# for p in pp:
#     print(p)


# model = "model/test_model_zg.npz"
# with open('%s.pkl'%model, 'rb') as f:
#     model_options = pkl.load(f)
# print(model_options)
# pp = numpy.load(model)
# for p in pp:
#     print(p)

100


In [ ]:
# test logging
# test 1, no print, 
# import logging

# logging.basicConfig(level=logging.INFO, format='%(message)s')
# logger = logging.getLogger()
# logger.addHandler(logging.FileHandler('test_1_log_zg.log', 'a'))

# print = logger.info

# print("hello!")

# test 2, no print
# log = open("test_2_log_zg.log", "a")
# sys.stdout = log
# print("hello!")


In [ ]:
# build dictionary from token lists
'''
def build_dictionary(filename):
    rows = pickle_load(filename)
    s = set(".")
    d = {}
    for row in rows:
        token_list = row[6]
        s.update(token_list)

    for i, ss in enumerate(s):
        d[ss] = i

    return d

D = build_dictionary("output_data/testing_masked_list.pkl")
# training: 13580; testing: 3946
print(len(D))
'''

'\ndef build_dictionary(filename):\n    rows = pickle_load(filename)\n    s = set(".")\n    d = {}\n    for row in rows:\n        token_list = row[6]\n        s.update(token_list)\n\n    for i, ss in enumerate(s):\n        d[ss] = i\n\n    return d\n\nD = build_dictionary("output_data/testing_masked_list.pkl")\n# training: 13580; testing: 3946\nprint(len(D))\n'

In [ ]:
# ========================================
# compare with BOW n-gram
# get labels from original data
# ========================================

# sample_size = 1400

def return_labels(filename):
    reader = csv.reader(open(filename, newline='', encoding='utf-8'), delimiter=',',
                        quotechar='"')
    row_cnt = 0
    labels = []

    try:
        for row in reader:
            row_cnt += 1
            if row_cnt == 1:
                continue

            # if row_cnt > sample_size:
            #     break
            labels.append(int(row[5]))

    except Exception as ex:
        sys.stderr.write('Exception\n')
        extype, exvalue, extrace = sys.exc_info()
        traceback.print_exception(extype, exvalue, extrace)

    return labels

def return_embs(filename):
    reader = csv.reader(open(filename, newline='', encoding='utf-8'), delimiter=',',
                        quotechar='"')
    row_cnt = 0
    embs = []

    try:
        for row in reader:
            row_cnt += 1
            emb_str = str(row[0])
            emb = np.fromstring(emb_str, sep = " ")
            embs.append(emb)

    except Exception as ex:
        sys.stderr.write('Exception\n')
        extype, exvalue, extrace = sys.exc_info()
        traceback.print_exception(extype, exvalue, extrace)

    return embs


from sklearn.feature_extraction.text import CountVectorizer


def my_tokenizer(s):
    return s.split()


def return_corpus(filename, variant):
    row_cnt = 0
    corpus_proc = []

    rows = pickle_load(filename)

    for row in rows:
        row_cnt += 1

        # if row_cnt > 1399:
        #     break

        text = row[1]
        if variant == "raw":
            text = " ".join(row[6])
        elif variant == "mask":
            text = " ".join(row[7])
        elif variant ==  "remove":
            text = " ".join([x for x in row[7] if x != "#"])
            
        corpus_proc.append(text)

    return corpus_proc


def return_bow(corpus_proc):
    bow = []
    vectorizer = CountVectorizer(tokenizer=my_tokenizer,ngram_range=(1,1),max_features=30000)
    X = vectorizer.fit_transform(corpus_proc)
    # print(vectorizer.get_feature_names())
    X_list = list(X.toarray()) # 2d numpy array
    
    for x in X_list:
        bow.append(np.array(x))

    return bow

# labels = return_labels("output_data/training_marked.csv")
# embs = return_embs("output_data/sdae_emb_corrected_1.txt")
sdae_test_labels = return_labels("output_data/testing_marked.csv")
sdae_test_embs = return_embs("output_data/sdae_emb_corrected_1_tmp.txt") #  sdae_emb_corrected_1_testset

sdae_train_labels = return_labels("output_data/training_marked.csv")
sdae_train_embs = return_embs("output_data/sdae_emb_corrected_1_train_tmp.txt")

# corpus_proc = return_corpus("output_data/training_masked_list.pkl", "mask")
# bow_emb = return_bow(corpus_proc)
# print(len(corpus_proc), len(bow_emb))
# print(len(labels), len(bow_emb))
# print(len(bow_emb[0]))

print_time("test: ", str(len(sdae_test_embs)), ", train: ", str(len(sdae_train_embs)))


[ 2019-12-09 22:43:11.103056 ] test: 704, train: 6328


In [ ]:
# validate embeddings
# input_list = pickle_load("output_data/testing_masked_list.pkl")
# print(len(input_list))

# print(len(sdae_test_embs))

# with open("output_data/testing_corpus.txt", 'r') as txtfile:
#     row_cnt = 0
#     for row in txtfile.readlines():
#         row_cnt+=1
#     print(row_cnt)

print(sdae_train_labels)

[1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 

In [ ]:
# ========================================
# compare pretrained embeddings and SDAE performance
# ========================================

# ------- ELMo --------
# elmo_train_pkl = pickle_load("output_data/Elmo_training_marked_vecs.txt.dict.pkl")
# elmo_train_pkl = pickle_load("output_data/Elmo_training_marked1_vecs.txt.dict.pkl")
elmo_train_pkl = pickle_load("output_data/Elmo_training_tokens1_vecs.txt.dict.pkl")
elmo_train_embs = [row[2] for row in elmo_train_pkl]
elmo_train_labels = [int(row[1]) for row in elmo_train_pkl]

# elmo_test_pkl = pickle_load("output_data/Elmo_testing_marked_vecs.txt.dict.pkl")
# elmo_test_pkl = pickle_load("output_data/Elmo_testing_marked1_vecs.txt.dict.pkl")
elmo_test_pkl = pickle_load("output_data/Elmo_testing_tokens1_vecs.txt.dict.pkl")
elmo_test_embs = [row[2] for row in elmo_test_pkl]
elmo_test_labels = [int(row[1]) for row in elmo_test_pkl]

elmo_clf = LogisticRegression(random_state=0, solver='liblinear').fit(elmo_train_embs, elmo_train_labels)
elmo_predict_labels = elmo_clf.predict(elmo_test_embs)

print(elmo_clf.score(elmo_test_embs, elmo_test_labels))
# print("elmo: ", elmo_predict_labels)
# marked: 0.796875; unmarked: 0.81


# ------- SDAE   --------
sdae_train_embs = return_embs("output_data/sdae_emb_corrected_1.txt") # sdae_emb_corrected_1 vs. sdae_emb_corrected_1_train_tmp 
sdae_test_embs = return_embs("output_data/sdae_emb_corrected_1_testset.txt") # sdae_emb_corrected_1_testset vs. sdae_emb_corrected_1_tmp

sdae_clf = LogisticRegression(random_state=0, solver='liblinear').fit(sdae_train_embs, elmo_train_labels)
sdae_predict_labels = sdae_clf.predict(sdae_test_embs)
# print("sdae: ", sdae_predict_labels)
print(sdae_clf.score(sdae_test_embs, sdae_test_labels))
# 0.609375
print(sdae_test_labels)

# ------- USE   --------
use_train_pkl = pickle_load("output_data/training_USE_tokens_vecs.txt.dict.pkl")
use_train_embs = [row[2][0] for row in use_train_pkl]
use_train_labels = [int(row[1]) for row in use_train_pkl]

use_test_pkl = pickle_load("output_data/testing_USE_tokens_vecs.txt.dict.pkl")
use_test_embs = [row[2][0] for row in use_test_pkl]
use_test_labels = [int(row[1]) for row in use_test_pkl]

use_clf = LogisticRegression(random_state=0, solver='liblinear').fit(use_train_embs, use_train_labels)
use_predict_labels = use_clf.predict(use_test_embs)

print(use_clf.score(use_test_embs, use_test_labels))


# ------- USE   --------

w2v_train_pkl = pickle_load("output_data/training_word_vec_tokens_vecs.txt.dict.pkl")
w2v_train_embs = [row[2] for row in w2v_train_pkl]
w2v_train_labels = [int(row[1]) for row in w2v_train_pkl]

w2v_test_pkl = pickle_load("output_data/testing_word_vec_tokens_vecs.txt.dict.pkl")
w2v_test_embs = [row[2] for row in w2v_test_pkl]
w2v_test_labels = [int(row[1]) for row in w2v_test_pkl]

w2v_clf = LogisticRegression(random_state=0, solver='liblinear').fit(w2v_train_embs, w2v_train_labels)
w2v_predict_labels = w2v_clf.predict(w2v_test_embs)

print(w2v_clf.score(w2v_test_embs, w2v_test_labels))
print_time()

0.8167613636363636
0.609375
[1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,

In [ ]:
# ========================================
# compare classifier performance
# ========================================

def get_binary_labels(orig_labels):
    threshold = 0.7
    bi_labels = [1 if x>threshold else 0 for x in orig_labels]
    return bi_labels


def calculate_auc_roc(test_label_trans, predict_label_trans):
    
    print('ROC-AUC: ', roc_auc_score(test_label_trans, predict_label_trans))
    fpr, tpr, thresholds = roc_curve(test_label_trans, predict_label_trans, pos_label=1)

    print(classification_report(test_label_trans, get_binary_labels(predict_label_trans)))

    roc_auc = metrics.auc(fpr, tpr)

    import matplotlib
    # matplotlib.use("TkAgg")
    matplotlib.rcParams.update({'font.size': 20})
    f=plt.figure()
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()


def calculate_ap(test_label_trans, predict_label_trans):
    average_precision = average_precision_score(test_label_trans, predict_label_trans)

    print('PRC for skip-chain CRF with sub-sequences \n {0:0.2f}'.format(
          average_precision))

    # plot PRC

    precision, recall, thresholds = precision_recall_curve(test_label_trans, predict_label_trans)

    pos_rate = sum(test_label_trans)/len(predict_label_trans)
#     import matplotlib
    f=plt.figure()
    matplotlib.rcParams.update({'font.size': 20})
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post', label='AP = %0.2f' % average_precision)
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)
    plt.plot([0, 1], [pos_rate, pos_rate], 'r--')
    plt.legend(loc='upper right')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    
    
def calculate_mrr(test_label, predict_result):
    ranked_results = []

    for i in range(len(predict_result)):
        list1 = test_label[i]
        list2 = predict_result[i]

        list2_ranked, list1_ranked = zip(*sorted(zip(list2,list1), reverse=True))
#         list1_ranked_int = list(map(int, list1_ranked))
        list1_ranked_int = list1_ranked
#         list1_ranked_int = get_binary_labels(list1_ranked)
        ranked_results.append(list1_ranked_int)
        if i<3:
            print(ranked_results)


    rs = (np.asarray(r).nonzero()[0] for r in ranked_results)

    mrr = np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

    print("mrr: ", mrr)


def run_lr(data_set, data_label, metrics):
    # print("----- LR -----")

    predicted = cross_validate(LogisticRegression(solver='liblinear'), data_set, data_label, cv=3, scoring=(metrics), return_train_score=True) #, class_weight='balanced'

    return predicted

# run_lr(embs, labels)
# predicted = run_lr(use_train_embs, train_labels, 'f1')
# run_lr(bow_emb,labels)
print_time()

[ 2019-12-09 03:36:47.023248 ] 


In [ ]:
# ========================================
# cross-validation results (a sense of overfitting)
# ========================================
# ELMo
elmo_train_pkl = pickle_load("output_data/Elmo_training_tokens1_vecs.txt.dict.pkl")
elmo_train_embs = [row[2] for row in elmo_train_pkl]
elmo_train_labels = [int(row[1]) for row in elmo_train_pkl]

# ELMo masked
elmo_mask_train_pkl = pickle_load("output_data/Elmo_training_marked1_vecs.txt.dict.pkl")
elmo_mask_train_embs = [row[2] for row in elmo_mask_train_pkl]
elmo_mask_train_labels = [int(row[1]) for row in elmo_mask_train_pkl]

# word2vec
w2v_train_pkl = pickle_load("output_data/training_word_vec_tokens_vecs.txt.dict.pkl")
w2v_train_embs = [row[2] for row in w2v_train_pkl]
w2v_train_labels = [int(row[1]) for row in w2v_train_pkl]

# word2vec masked
w2v_mask_train_pkl = pickle_load("output_data/training_word_vec_marked_vecs.txt.dict.pkl")
w2v_mask_train_embs = [row[2] for row in w2v_mask_train_pkl]
w2v_mask_train_labels = [int(row[1]) for row in w2v_mask_train_pkl]

# USE sentence encoding
use_train_pkl = pickle_load("output_data/training_USE_tokens_vecs.txt.dict.pkl")
use_train_embs = [row[2][0] for row in use_train_pkl]
use_train_labels = [int(row[1]) for row in use_train_pkl]

# USE masked
use_mask_train_pkl = pickle_load("output_data/training_USE_marked_vecs.txt.dict.pkl")
use_mask_train_embs = [row[2][0] for row in use_mask_train_pkl]
use_mask_train_labels = [int(row[1]) for row in use_mask_train_pkl]


examine_data = {
    "elmo": (elmo_train_embs, elmo_train_labels),
    "elmo_mask": (elmo_mask_train_embs, elmo_mask_train_labels),
    "use": (use_train_embs, use_train_labels),
    "use_mask": (use_mask_train_embs, use_mask_train_labels),
    "w2v": (w2v_train_embs, w2v_train_labels),
    "w2v_mask": (w2v_mask_train_embs, w2v_mask_train_labels),
    "sdae": (sdae_train_embs, sdae_train_labels)
}

# for key in examine_data.keys():
#     embs = examine_data.get(key)[0]
#     labels = examine_data.get(key)[1]
#     print("----- ", key) 
#     for m in ('f1', 'accuracy', 'precision', 'roc_auc'):
#         predicted = run_lr(embs, labels, m)
#         # predicted = run_lr(sdae_train_embs, sdae_train_labels, m)
#         mean_score = np.mean(predicted.get("test_score"))
#         mean_score_training = np.mean(predicted.get("train_score"))
#         print(mean_score, mean_score_training)

print_time()

[ 2019-12-09 03:36:55.977632 ] 


In [ ]:
# test

for key in ["w2v", "use", "elmo", "sdae"]:
    print("----- ", key)
    embs = examine_data.get(key)[0]
    labels = examine_data.get(key)[1]
    for m in ('f1', 'accuracy', 'precision', 'roc_auc'):
        predicted = run_lr(embs, labels, m)
        # predicted = run_lr(sdae_train_embs, sdae_train_labels, m)
        mean_score = np.mean(predicted.get("test_score"))
        mean_score_training = np.mean(predicted.get("train_score"))
        drop = mean_score_training - mean_score
        print(m, mean_score, mean_score_training, drop)

-----  w2v
f1 0.7504103950856919 0.7647922439137401 0.014381848828048271
accuracy 0.7731238429081233 0.7864823295126184 0.013358486604495123
precision 0.7518717803990639 0.7674302564928247 0.01555847609376082
roc_auc 0.8355330862339626 0.8590766958943806 0.02354360966041802
-----  use
f1 0.7776290057183534 0.7918041865177141 0.014175180799360754
accuracy 0.7948806626531745 0.8081544075730775 0.013273744919903074
precision 0.767658137010642 0.7825464367533507 0.014888299742708666
roc_auc 0.865522917630798 0.8827731702889371 0.017250252658139065
-----  elmo
f1 0.7380900514074801 0.798253806078156 0.06016375467067592
accuracy 0.7779731340220241 0.8269600484580337 0.048986914436009665
precision 0.7957626266000482 0.8505953511904445 0.05483272459039634
roc_auc 0.8493229493861306 0.9093459053994032 0.060022956013272566
-----  sdae
f1 0.7290319866532248 0.7313504919585566 0.0023185053053318416
accuracy 0.754424616684532 0.7560840115573714 0.0016593948728393926
precision 0.7325594909678621 0.7

In [ ]:
# ========================================
# get top n similiar sentences:
# ========================================
import heapq 
from sklearn.metrics.pairwise import cosine_similarity
import scipy

# def get_idx_sentence_mapping(idx)
idx_sent_map = {}
idx_entry_map = {}
raw_list = pickle_load("output_data/training_masked_list.pkl")

SENTENCE_ID_COL = 0
SENTENCE_COL = 1
RAW_SPLIT_COL = 6
MASK_SPLIT_COL = 7

num_top = 3

for idx,r in enumerate(raw_list):
    idx_sent_map[idx] = (r[0],r[1])
    idx_entry_map[idx] = r


def get_n_similar(embs, input_idx, n):
    dist_list = []
    # heapq.heapify(simi_heapq)
    input_emb = embs[input_idx]

    for idx, vec in enumerate(embs):
        if idx == input_idx:
            continue
        
        dist = scipy.spatial.distance.cosine(input_emb, vec)        
        dist_list.append((dist,idx))

    # print("input: ", input_idx, idx_sent_map.get(input_idx))
    top_simi = []
    for i in heapq.nsmallest(n,dist_list):
        dist = i[0]
        idx = i[1]
        sent_id = idx_entry_map.get(idx)[SENTENCE_ID_COL]
        sent = idx_entry_map.get(idx)[SENTENCE_COL]
        entry = (idx, sent_id, sent, dist)
        top_simi.append(entry)
        # print("list idx", i[1], "| (id, sentence):", idx_sent_map.get(i[1]))
    return top_simi # heapq.nsmallest(n,dist_list)

sample_idx = 737

# sample training idx 737, id 3833, "My village Town Hall built with a huge sum of money has been swallowed," he said.
# sample training idx 0, id 2864, In the last four years, the Maoists have killed more than 900 Indian security officers, a figure almost as high as the more than 1,100 members of the coalition forces killed in Afghanistan during the same period.

print("input: ", idx_entry_map.get(sample_idx)[SENTENCE_COL])

for key in ["elmo", "elmo_mask", "w2v", "w2v_mask", "use", "use_mask", "sdae"]:    #["elmo", "elmo_mask", "w2v", "w2v_mask", "use", "use_mask", "sdae"]:
    print("------------ ", key)
    embs = examine_data.get(key)[0]
    top_simi = get_n_similar(embs,sample_idx,num_top)
    print("top_simi: ", top_simi)

print_time()


input:  "My village Town Hall built with a huge sum of money has been swallowed," he said.
------------  elmo
top_simi:  [(667, '6544', '"We have a shortage of resources," he said.', 0.17784839868545532), (1039, '6565', '"We just heard this real roar, I thought we were about to lose half our roof," she said.', 0.18492305278778076), (3600, '5176', '"The arrival of Mr. Shannon was key," he said.', 0.18993449211120605)]
------------  elmo_mask
top_simi:  [(2881, '1485', 'Eschol Amelia "Amy" Studnitz had been working for Corporate Mailing Services since August 2008 as a senior accountant.', 0.05905866622924805), (4926, '6328', "Treece said O'Quinn's personal property has been left to the foundation that served his charitable giving.", 0.060838162899017334), (6030, '225', 'A large debris field has been located, and debris has been collected, he said.', 0.061478376388549805)]
------------  w2v
top_simi:  [(3448, '491', 'Another man arrived on a motorcycle from a more distant village.', 0.103

In [ ]:
# test
sample_idx = 737
num_top = 3
tmp_embs = examine_data.get("elmo")[0]
top_simi = get_n_similar(tmp_embs,sample_idx,num_top)
print("top_simi: ", top_simi)
    

top_simi:  [(667, '6544', '"We have a shortage of resources," he said.', 0.17784839868545532), (1039, '6565', '"We just heard this real roar, I thought we were about to lose half our roof," she said.', 0.18492305278778076), (3600, '5176', '"The arrival of Mr. Shannon was key," he said.', 0.18993449211120605)]


In [ ]:

# if __name__ == "__main__":
#     import doctest
#     doctest.testmod()

# sample_idx = 737

raw_masked_split = idx_entry_map.get(sample_idx)[MASK_SPLIT_COL]

# save to pickle wer_result_dict = {"elmo": [{"input": (idx, id, sentence), "selected_1": (idx,id,sentence,dist,wer), "selected_2": , "selected_3": , "avg_wer": top3_avg_wer_rate)}],
            # "use":  [{"input": (idx, id, sentence), "selected_1": , "selected_2": , "selected_3": , "avg_wer": top3_avg_wer_rate)},{...}, ...] ...

print_time("calculating wer...")
wer_result_dict = {}
for key in ["use"]:     # ["elmo", "elmo_mask", "w2v", "w2v_mask", "use", "use_mask", "sdae"]
    sum_wer_by_key = 0
    wer_result_dict[key] = []
    for sample_idx in range(0,len(idx_entry_map)):
        if sample_idx % 2000 == 0:
            print_time(key, ", ", str(sample_idx))
        embs = examine_data.get(key)[0]
        input_id = idx_entry_map.get(sample_idx)[SENTENCE_ID_COL]
        input_sent = idx_entry_map.get(sample_idx)[SENTENCE_COL]
        sample_input = (sample_idx, input_id, input_sent)

        top_simi = get_n_similar(embs,sample_idx,num_top)
        top3_sum_wer_rate = 0.0
        # print("------ ", key)
        selected_entry = []
        for i, (idx, sent_id, sent, dist) in enumerate(top_simi):
            selected_masked_split = idx_entry_map.get(idx)[MASK_SPLIT_COL]
            wer_dist = adapted_wer(raw_masked_split, selected_masked_split)
            # print(raw_masked_split)
            # print(selected_masked_split)
            # print(wer_dist)
            wer_rate = wer_dist / len(raw_masked_split)
            selected_entry.append((idx, sent_id, sent, dist, wer_rate))
            top3_sum_wer_rate += wer_rate

        top3_avg_wer_rate = top3_sum_wer_rate / 3.0
        this_sample = {"input": sample_input,
                       "selected_1": selected_entry[0],
                       "selected_2": selected_entry[1],
                       "selected_3": selected_entry[2],
                       "avg_wer": top3_avg_wer_rate
        }
        wer_result_dict.get(key).append(this_sample)
        # print(key, ": ", top3_avg_wer_rate)
        sum_wer_by_key += top3_avg_wer_rate
    avg_wer_by_key = sum_wer_by_key / len(idx_entry_map)    
    print(key,  avg_wer_by_key)    

print_time("done")

# [ 2019-12-06 21:21:07.547086 ] calculating wer...
# [ 2019-12-06 21:21:07.558269 ] use, 0
# [ 2019-12-06 21:34:30.121963 ] use, 2000
# [ 2019-12-06 21:47:58.976531 ] use, 4000
# [ 2019-12-06 22:01:20.676744 ] use, 6000
# use 0.46128061628279626
# [ 2019-12-06 16:52:20.933022 ] calculating wer...
# [ 2019-12-06 16:52:20.934027 ] use_mask, 0
# [ 2019-12-06 17:05:45.124735 ] use_mask, 2000
# [ 2019-12-06 17:19:10.318847 ] use_mask, 4000
# [ 2019-12-06 17:32:31.875477 ] use_mask, 6000
# use_mask 0.6143357564264627
# [ 2019-12-06 17:34:42.980073 ] sdae, 0
# [ 2019-12-06 17:43:21.121699 ] sdae, 2000
# [ 2019-12-06 17:52:00.575388 ] sdae, 4000
# [ 2019-12-06 18:00:39.874949 ] sdae, 6000
# sdae 0.6306218394437425
# [ 2019-12-06 18:02:04.642694 ] done

[ 2019-12-06 21:21:07.547086 ] calculating wer...
[ 2019-12-06 21:21:07.558269 ] use, 0
[ 2019-12-06 21:34:30.121963 ] use, 2000
[ 2019-12-06 21:47:58.976531 ] use, 4000
[ 2019-12-06 22:01:20.676744 ] use, 6000
use 0.46128061628279626
[ 2019-12-06 22:03:31.509665 ] done


In [ ]:
# merge single files to wer_result_dict

# wer_result_dict = {}

# elmo_selects = pickle_load("selected_elmo.pkl")
# wer_result_dict["elmo"] = elmo_selects

# dict_reload = pickle_load("selected_use.pkl")
# wer_result_dict["use"] = dict_reload.get("use")

# dict_reload = pickle_load("selected_use_mask_sdae.pkl")
# wer_result_dict["use_mask"] = dict_reload.get("use_mask")
# wer_result_dict["sdae"] = dict_reload.get("sdae")

# dict_reload = pickle_load("selected_elmo_mask_w2v.pkl")
# wer_result_dict["elmo_mask"] = dict_reload.get("elmo_mask")
# wer_result_dict["w2v"] = dict_reload.get("w2v")

# dict_reload = pickle_load("selected_w2v_mask.pkl")
# wer_result_dict["w2v_mask"] = dict_reload.get("w2v_mask")

# pickle_save("analysis/all_selected_wer.pkl", wer_result_dict)


In [ ]:
# ========================================
# reload wer_result_dict
# ========================================

wer_result_dict =  pickle_load("analysis/all_selected_wer.pkl")
print(wer_result_dict.keys())


dict_keys(['elmo', 'use', 'use_mask', 'sdae', 'elmo_mask', 'w2v', 'w2v_mask'])


In [ ]:
# measure with WER(word error rate) for structual similarity
# edited from : https://martin-thoma.com/word-error-rate-calculation/
# with Hunt (1990) has proposed the use of a weighted measure of performance where deletion and insertion are both weighted only at 0.5
def adapted_wer(r, h):
    """
    r is reference, h is hypothesis
    mask non-function words with "#" when calculate distance
    """
    # initialisation
    import numpy
    d = numpy.zeros((len(r)+1)*(len(h)+1)) 
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            inc_sub = 1
            inc_ins = 0.2
            inc_del = 0.2
            # if r[i-1] == "#":
            #     inc_sub, inc_ins, inc_del = 0, 0.5, 0.5
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + inc_sub
                insertion    = d[i][j-1] + inc_ins
                deletion     = d[i-1][j] + inc_del
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]



In [ ]:


def recalculate_avg_wer(selects):
    sum_wer = 0 
    for select in selects:
        selected_1 = select.get("selected_1")
        selected_2 = select.get("selected_2")
        selected_3 = select.get("selected_3")

        sum_wer += (selected_1[4]+selected_2[4] + selected_3[4])/3
        # sum_wer += selected_1[4]
    return sum_wer/len(selects)

def recalculate_wer(selects):
    sum_all_wer = 0
    for select in selects:
        raw_masked_split = select.get('input')

        input_idx = select.get("input")[0]
        raw_masked_split = idx_entry_map.get(input_idx)[MASK_SPLIT_COL]
        sum_select_wer = 0
        for i in ["1","2", "3"]:
            select_idx =  select.get("selected_"+i)[0]
            selected_masked_split = idx_entry_map.get(select_idx)[MASK_SPLIT_COL]
            wer_dist = adapted_wer(raw_masked_split, selected_masked_split)
            wer_rate = wer_dist/len(raw_masked_split)
            sum_select_wer += wer_rate
        avg_wer = sum_select_wer/3
        sum_all_wer += avg_wer

    return sum_all_wer/len(selects)

# for key, selects in wer_result_dict.items():
#     print(key, recalculate_avg_wer(selects))

# sub =1, ins = 1, del = 1, recover from reloaded data
# elmo 0.5042090582545696
# use 0.46128061628279626
# use_mask 0.6143357564264627
# sdae 0.6306218394437425
# elmo_mask 0.6856460360211806
# w2v 0.6610528727731038
# w2v_mask 2.4222398729150205



# for key, selects in wer_result_dict.items():
#     print(key, recalculate_wer(selects))


# sub = 1, ins = 0.5, del = 0.5
# elmo 0.5402552829417606
# use 0.6391554046335199
# use_mask 0.5448933146043207
# sdae 0.5004732376210101
# elmo_mask 0.48619971794627437
# w2v 0.905774806031501
# w2v_mask 0.920564334921754

# for key, selects in wer_result_dict.items():
    # print(key, recalculate_wer(selects))

    
# sub = 1, ins = 0.1, del = 0.1
# elmo 0.1442342006113179
# use 0.17527708765817168
# use_mask 0.1453520523694472
# sdae 0.1226655331718089
# elmo_mask 0.12327695719821678
# w2v 0.29092100592104553
# w2v_mask 0.29132159684799863



elmo 0.2435208530816369
use 0.29149197389850523
use_mask 0.24572459543023525
sdae 0.21725605188418745
elmo_mask 0.21424600794927992
w2v 0.4447332373107591
w2v_mask 0.4487530993702097


In [ ]:
tmp_exm_idx = 6327
print(wer_result_dict.get("elmo")[tmp_exm_idx])
print(wer_result_dict.get("elmo_mask")[tmp_exm_idx])
print(wer_result_dict.get("use_mask")[tmp_exm_idx])
print(wer_result_dict.get("sdae")[tmp_exm_idx])

{'input': (6327, '3005', '"It could be a little choppy. It could be uneven. And it\'s going to take awhile."'), 'selected_1': (726, '3248', "It's possible, it's just going to look ugly unless you want to install the rest of KDE, that's all.", 0.14808011054992676, 0.42857142857142855), 'selected_2': (1513, '6326', 'Treasury Secretary Timothy Geithner says the economic recovery "could be a little choppy" and it\'s going to take a while.', 0.14885741472244263, 0.4880952380952381), 'selected_3': (3321, '6130', '"They\'re just so beneficial right now. It\'s a luxury right now. It takes so much pressure off us."', 0.1510900855064392, 0.5833333333333334), 'avg_wer': 0.5}
{'input': (6327, '3005', '"It could be a little choppy. It could be uneven. And it\'s going to take awhile."'), 'selected_1': (5369, '3006', "It could be done to a change of frequency, but it should report it...( thing that doesn't happens).", 0.05625051259994507, 0.5), 'selected_2': (1097, '639', '"As awful as this attack wa

In [ ]:
# ========================================
# generate results_dist.csv

import csv

with open("analysis/results_dist.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     my_header = ["input_idx", "input_sent_id", "input_sent",
                  "elmo_idx", "elmo_sent_id", "elmo_sent","elmo_dist",
                   "emlo_mask_idx", "emlo_mask_sent_id", "emlo_mask_sent","emlo_mask_dist",
                   "use_idx", "use_sent_id", "use_sent","use_dist",
                   "use_mask_idx", "use_mask_sent_id", "use_mask_sent","use_mask_dist",
                  "sdae_idx", "sdae_sent_id", "sdae_sent","sdae_dist",
                  "num_candidate"
                  ]
     wr.writerow(my_header)
     my_list = []
     for idx in range(0,6328):
        my_item = []
        input_sample = wer_result_dict.get("elmo")[idx].get("input")
        my_item.append(input_sample[0])
        my_item.append(input_sample[1])
        my_item.append(input_sample[2])
        candidate_set = set()
        for item_key in ["elmo","elmo_mask","use","use_mask","sdae"]:
            this_top = wer_result_dict.get(item_key)[idx].get("selected_1")
            my_item.append(this_top[0])
            my_item.append(this_top[1])
            my_item.append(this_top[2])
            my_item.append(this_top[3])
            candidate_set.add(this_top[0])
        num_candidate = len(candidate_set)
        my_item.append(num_candidate)
        my_list.append(my_item)
        wr.writerow(my_item)

pickle_save("analysis/results_dist.pkl", my_list)

print(len(my_list))
print(my_list[2])


6328
[2, '588', 'A reading above 50 indicates an expansion.', 1833, '4435', 'Readings greater than 50 signal expansion.', 0.10464566946029663, 1251, '130', 'A couple of Marriotts just opened.', 0.048340022563934326, 1833, '4435', 'Readings greater than 50 signal expansion.', 0.34535109996795654, 5776, '6035', "The video, sadly, winks out right as the expert is being brought in to explain Skinner's research.", 0.1761012077331543, 784, '5181', 'The authors report no financial disclosures.', 0.0009925732658990771, 4]


In [ ]:
# ========================================
# reformat for human evaluation from mylist

my_list = pickle_load("analysis/results_dist.pkl")

candidates = []
# input_idx_list = [0,2,3,8,30,37,38,40,42,47,52,56,58,62,66,69,71,83,90,93]
# input_idx_list = [99,101,110,116,118,122,124,129,135,137,147,159,160,168,184,187,197,209,211,216]
input_idx_list = [218,231,235,238,241,258,285,303,305,322,329,330,333,335,336,352,357,366,376,379]
                  
for idx in input_idx_list:
    item = my_list[idx]
    # candidate_set = set()
    # candidate_set.add(item[5])
    # candidate_set.add(item[9])
    # candidate_set.add(item[13])
    # candidate_set.add(item[17])
    # candidate_set.add(item[21])
        
    candidate_dict = {}
    candidate_dict["input"] = item[2]
    candidate_dict[item[3]] = item[5]
    candidate_dict[item[7]] = item[9]
    candidate_dict[item[11]] = item[13]
    candidate_dict[item[15]] = item[17]
    candidate_dict[item[19]] = item[21]

    if len(candidate_dict) != 5:
        print("wrong", idx, len(candidate_dict))
    candidates.append(candidate_dict)



with open("analysis/human_eval_c4_f41-60.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    # wr.writerow(["input","selected_1", "selected_2", "selected_3", "selected_4"])
    wr.writerow(["input_candidates", "rater_selected"])
    for cand in candidates:
        formatted = "input: " + cand.get("input") + "\n"
        for key, value in cand.items():
            if key != "input":
                formatted += str(key) + ". " + value + "\n"
        wr.writerow([formatted, ""])


In [ ]:
# ========================================
# examine raters selections
# ========================================

# reader = csv.reader(open("analysis/human_eval_results_v0.csv", newline='', encoding='utf-8'), delimiter=',',
#                         quotechar='"')
my_list = pickle_load("analysis/results_dist.pkl")

reader = csv.reader(open("analysis/human_eval_results_f60.csv", newline='', encoding='utf-8'), delimiter=',',
                        quotechar='"')
row_cnt = 0

key_idxcol_map = {"elmo": 3, "elmo_mask": 7, "use": 11, "use_mask": 15, "sdae": 19}

match_list = []
match_cnt = {"elmo":0, "elmo_mask": 0, "use": 0, "use_mask": 0, "sdae": 0}

selection_dict = {
    "rater1": [],
    "rater2": [],
    "elmo": [],
    "elmo_mask": [],
    "use": [],
    "use_mask": [],
    "sdae": [],
    "reference": []
}

try:
    for row in reader:
        row_cnt += 1
        if row_cnt == 1:
            continue
        
        input_idx = int(row[0])
        rater1_idx = int(row[1])
        rater2_idx = int(row[2])

        selection_dict.get("rater1").append(rater1_idx)
        selection_dict.get("rater2").append(rater2_idx)
        selection_dict.get("reference").append(set([rater1_idx,rater2_idx]))

        record = my_list[input_idx]
        match = []
        for key,value in key_idxcol_map.items():
            selected_idx = int(record[value])
            selection_dict.get(key).append(selected_idx)
            if selected_idx == rater1_idx or selected_idx == rater2_idx:
                match.append(key)
                match_cnt[key] = match_cnt[key] + 1

        # print(match)

        match_list.append(match)

except Exception as ex:
    sys.stderr.write('Exception\n')
    extype, exvalue, extrace = sys.exc_info()
    traceback.print_exception(extype, exvalue, extrace)

# print(match_list)
print(match_cnt)
print([(key, value/40) for key, value in match_cnt.items()])
# print(selection_dict)
# print(selection_dict.get("rater1"))
# print(selection_dict.get("elmo"))

# f40:
# {'elmo': 10, 'elmo_mask': 14, 'use': 8, 'use_mask': 5, 'sdae': 25}
# [('elmo', 0.25), ('elmo_mask', 0.35), ('use', 0.2), ('use_mask', 0.125), ('sdae', 0.625)]





{'elmo': 17, 'elmo_mask': 23, 'use': 14, 'use_mask': 7, 'sdae': 35}
[('elmo', 0.425), ('elmo_mask', 0.575), ('use', 0.35), ('use_mask', 0.175), ('sdae', 0.875)]


In [ ]:
from statsmodels.stats.contingency_tables import mcnemar

def mcnemar_pvalue(list1,list2,truth_list):
    cnt_yes_yes = 0
    cnt_yes_no = 0
    cnt_no_yes = 0
    cnt_no_no = 0

    yes_yes, yes_no, no_yes, no_no = [], [], [], []

    # calculate contingency table
    # format: contingency_table = [[cnt_yes_yes, cnt_yes_no],[cnt_no_yes, cnt_no_no]]

    for i in range(len(list1)):
        truth = truth_list[i]
        label_1 = list1[i]
        label_2 = list2[i]

        if truth_list is not None:
            # --------------------------------
            # compute correctness
            if label_1 in truth and label_2 in truth:
                cnt_yes_yes += 1      
            elif label_1 in truth and label_2 not in truth:
                cnt_yes_no += 1
            elif label_1 not in truth and label_2 in truth:
                cnt_no_yes += 1
            elif label_1 not in truth and label_2 not in truth:
                cnt_no_no += 1
            else:
                print("correctness comparison error")
        else:
            # --------------------------------
            # compute agreement
            if label_1 == 1 and label_2 == 1:
                cnt_yes_yes += 1  
                yes_yes.append(i)    
            elif label_1 == 1 and label_2 != 1:
                cnt_yes_no += 1
                yes_no.append(i)
            elif label_1 != 1 and label_2 == 1:
                cnt_no_yes += 1
                no_yes.append(i)
            elif label_1 != 1 and label_2 != 1:
                cnt_no_no += 1
                no_no.append(i)
            else:
                print("agreement comparison error")
            
    contingency_table = [[cnt_yes_yes, cnt_yes_no],[cnt_no_yes, cnt_no_no]]
    print("contingency table: ", contingency_table)

    # calculate mcnemar test
    result = mcnemar(contingency_table, exact=True)

    # summarize the finding
    print('statistic=%.3f, p-value=%.2E' % (result.statistic, result.pvalue))

    # interpret the p-value
    alpha = 0.05

    if result.pvalue > alpha:
        print('Same proportions of errors (fail to reject H0)')
    else:
        print('Different proportions of errors (reject H0)')

    return result

print("------ elmo vs. elmo mask:")
p = mcnemar_pvalue(selection_dict.get("elmo"), selection_dict.get("elmo_mask"), selection_dict.get("reference"))
# print(p)

print("------ use vs. use mask:")
p = mcnemar_pvalue(selection_dict.get("use"), selection_dict.get("use_mask"), selection_dict.get("reference"))
# print(p)


print("------ sdae vs. elmo:")
p = mcnemar_pvalue(selection_dict.get("sdae"), selection_dict.get("elmo"), selection_dict.get("reference"))
# print(p)


print("------ sdae vs. use:")
p = mcnemar_pvalue(selection_dict.get("sdae"), selection_dict.get("use"), selection_dict.get("reference"))
# print(p)

print("------ sdae vs. elmo mask:")
p = mcnemar_pvalue(selection_dict.get("sdae"), selection_dict.get("elmo_mask"), selection_dict.get("reference"))
# print(p)


print("------ sdae vs. use mask:")
p = mcnemar_pvalue(selection_dict.get("sdae"), selection_dict.get("use_mask"), selection_dict.get("reference"))
# print(p)

# ------ elmo vs. elmo mask:
# contingency table:  [[3, 4], [4, 9]]
# statistic=4.000, p-value=1.00E+00
# Same proportions of errors (fail to reject H0)
# ------ use vs. use mask:
# contingency table:  [[0, 3], [1, 16]]
# statistic=1.000, p-value=6.25E-01
# Same proportions of errors (fail to reject H0)
# ------ sdae vs. elmo:
# contingency table:  [[2, 11], [5, 2]]
# statistic=5.000, p-value=2.10E-01
# Same proportions of errors (fail to reject H0)
# ------ sdae vs. use:
# contingency table:  [[1, 12], [2, 5]]
# statistic=2.000, p-value=1.29E-02
# Different proportions of errors (reject H0)
# ------ sdae vs. elmo mask:
# contingency table:  [[2, 11], [5, 2]]
# statistic=5.000, p-value=2.10E-01
# Same proportions of errors (fail to reject H0)
# ------ sdae vs. use mask:
# contingency table:  [[0, 13], [1, 6]]
# statistic=1.000, p-value=1.83E-03
# Different proportions of errors (reject H0)

------ elmo vs. elmo mask:
contingency table:  [[4, 6], [10, 20]]
statistic=6.000, p-value=4.54E-01
Same proportions of errors (fail to reject H0)
------ use vs. use mask:
contingency table:  [[1, 7], [4, 28]]
statistic=4.000, p-value=5.49E-01
Same proportions of errors (fail to reject H0)
------ sdae vs. elmo:
contingency table:  [[3, 22], [7, 8]]
statistic=7.000, p-value=8.13E-03
Different proportions of errors (reject H0)
------ sdae vs. use:
contingency table:  [[2, 23], [6, 9]]
statistic=6.000, p-value=2.32E-03
Different proportions of errors (reject H0)
------ sdae vs. elmo mask:
contingency table:  [[5, 20], [9, 6]]
statistic=9.000, p-value=6.14E-02
Same proportions of errors (fail to reject H0)
------ sdae vs. use mask:
contingency table:  [[2, 23], [3, 12]]
statistic=3.000, p-value=8.80E-05
Different proportions of errors (reject H0)


In [ ]:
# ========================================
# examine raters agreement
# ========================================

from sklearn.metrics import cohen_kappa_score


cohen_kappa_score(selection_dict.get("rater1"), selection_dict.get("rater2"))



0.7804054054054055

In [ ]:
# mcnemar test

# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar

cnt_yes_yes = 0
cnt_yes_no = 0
cnt_no_yes = 0
cnt_no_no = 0

yes_yes, yes_no, no_yes, no_no = [], [], [], []

# calculate contingency table
# format: contingency_table = [[cnt_yes_yes, cnt_yes_no],[cnt_no_yes, cnt_no_no]]

for i, label in enumerate(test_labels):
    truth = int(label)
    elmo_label = int(use_predict_labels[i])
    sdae_label = int(sdae_predict_labels[i])
    # print(elmo_label, sdae_label)
    # --------------------------------
    # compute correctness
    # if elmo_label == truth and sdae_label == truth:
    #     cnt_yes_yes += 1      
    # elif elmo_label == truth and sdae_label != truth:
    #     cnt_yes_no += 1
    # elif elmo_label != truth and sdae_label == truth:
    #     cnt_no_yes += 1
    # elif elmo_label != truth and sdae_label != truth:
    #     cnt_no_no += 1
    # else:
    #     print("Comparison error")
    
    # --------------------------------
    # compute agreement
    if elmo_label == 1 and sdae_label == 1:
        cnt_yes_yes += 1  
        yes_yes.append(i)    
    elif elmo_label == 1 and sdae_label != 1:
        cnt_yes_no += 1
        yes_no.append(i)
    elif elmo_label != 1 and sdae_label == 1:
        cnt_no_yes += 1
        no_yes.append(i)
    elif elmo_label != 1 and sdae_label != 1:
        cnt_no_no += 1
        no_no.append(i)
    else:
        print("Comparison error")
        
contingency_table = [[cnt_yes_yes, cnt_yes_no],[cnt_no_yes, cnt_no_no]]
print("contingency table: ", contingency_table)

# calculate mcnemar test
result = mcnemar(contingency_table, exact=True)

# summarize the finding
print('statistic=%.3f, p-value=%.2E' % (result.statistic, result.pvalue))

# interpret the p-value
alpha = 0.05

if result.pvalue > alpha:
	print('Same proportions of errors (fail to reject H0)')
else:
	print('Different proportions of errors (reject H0)')


print("Elmo yes SDAE no:", yes_no)
print("Elmo no Sdae yes:", no_yes)

contingency table:  [[89, 238], [43, 334]]
statistic=43.000, p-value=6.90E-34
Different proportions of errors (reject H0)
Elmo yes SDAE no: [1, 2, 3, 7, 15, 16, 23, 24, 30, 32, 34, 38, 39, 41, 43, 44, 48, 51, 55, 58, 63, 64, 67, 69, 71, 75, 79, 81, 82, 83, 86, 95, 96, 98, 100, 106, 107, 109, 110, 113, 115, 116, 117, 123, 124, 128, 130, 131, 135, 145, 146, 147, 148, 150, 153, 155, 156, 159, 160, 162, 164, 166, 169, 172, 173, 175, 180, 189, 192, 197, 198, 204, 205, 206, 210, 211, 214, 223, 227, 228, 231, 232, 235, 241, 243, 244, 249, 251, 253, 256, 257, 259, 260, 264, 269, 270, 276, 277, 278, 288, 289, 291, 292, 295, 301, 303, 304, 305, 310, 317, 321, 325, 331, 333, 334, 336, 337, 339, 342, 346, 347, 349, 350, 354, 357, 363, 366, 367, 372, 373, 378, 379, 380, 381, 382, 384, 389, 390, 391, 396, 398, 400, 401, 403, 405, 407, 408, 413, 416, 417, 418, 419, 421, 426, 427, 428, 429, 430, 432, 433, 434, 435, 441, 443, 445, 451, 459, 463, 466, 471, 473, 474, 475, 479, 482, 485, 486, 487, 488, 49

In [ ]:
print(np.mean([0.7137931 , 0.73231466, 0.76816239]))
print(np.mean([0.72485365, 0.72      , 0.74170124]))


0.7380900499999999
0.72885163


In [ ]:
# uni-gram
print("uni-gram")
# raw d-max (6102)
print(np.mean([0.73571429, 0.72142857, 0.73571429, 0.75      , 0.74193548]))
# raw d-1000
print(np.mean([0.71785714, 0.69642857, 0.72142857, 0.68571429, 0.72043011]))
# mask
print(np.mean([0.73928571, 0.72857143, 0.72142857, 0.73214286, 0.74193548]))

# bi-gram
print("bi-gram")
# raw
print(np.mean([0.63928571, 0.71428571, 0.65714286, 0.61785714, 0.68100358]))
# mask
print(np.mean([0.71428571, 0.75714286, 0.71428571, 0.69642857, 0.74551971]))

# -----------------------------

print("when mask all content words for learning embedding:")
# word-emb-d: 300, learned-d: 100
print(np.mean([0.69285714, 0.76071429, 0.725     , 0.71785714, 0.6953405 ]))


uni-gram
0.736958526
0.708371736
0.73267281
bi-gram
0.661915
0.725532512
when mask all content words for learning embedding:
0.718353814


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
 
# import tensorflow_hub as hub
# import tensorflow as tf
 
# elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
 
 
# def elmo_vectors(x): 
#   embeddings=elmo(x, signature="default", as_dict=True)["elmo"]
  
#   with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.tables_initializer())
#     # return average of ELMo features
#     return sess.run(tf.reduce_mean(embeddings,1))


corpus=['"The root of this is dispossession and deprivation," said Ramachandra Guha, a prominent historian based in Bangalore.',
'The symptoms were pretty much the same as what you describe; in fact, in particular, it was the alt key that had problems.',
'''And she's "learned what's important in life?"''',
'''Emanuel continues to believe that oceans in a warming world will produce stronger storms, because the trend over the last 30 years in the Western North Pacific and Atlantic basins is upward.''',
'''After another crushing loss, Washington State coach Paul Wulff took away the almost bowl-like atmosphere as at least one positive.''',
'''In its internal announcement, I.B.M. said Mr. Moffat was being replaced by Rodney C. Adkins, a rising younger executive, who becomes a senior vice president of the systems and technology group.'''
]

elmo_embeddings=[]

for i in range(len(corpus)):
    print (corpus[i])
    elmo_embeddings.append(elmo_vectors([corpus[i]])[0])

print (elmo_embeddings)
print(cosine_similarity(elmo_embeddings, elmo_embeddings))

"The root of this is dispossession and deprivation," said Ramachandra Guha, a prominent historian based in Bangalore.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


The symptoms were pretty much the same as what you describe; in fact, in particular, it was the alt key that had problems.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


And she's "learned what's important in life?"
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

After another crushing loss, Washington State coach Paul Wulff took away the almost bowl-like atmosphere as at least one positive.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In its internal announcement, I.B.M. said Mr. Moffat was being replaced by Rodney C. Adkins, a rising younger executive, who becomes a senior vice president of the systems and technology group.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[array([-0.3644273 , -0.35249865,  0.13963749, ..., -0.40541363,
        0.28308302, -0.04877667], dtype=float32), array([ 0.01186131,  0.35947883, -0.21745291, ..., -0.02570024,
        0.29493195,  0.1771647 ], dtype=float32), array([-0.00352885, -0.12127738,  0.34970155, ..., -0.17586514,
        0.42037353,  0.12657686], dtype=float32), array([ 0.15438657,  0.09934765,  0.1578552 , ..., -0.02052657,
        0.63652134,  0.18594106], dtype=float32), array([-0.10641526, -0.19809897,  0.05857019, ..., -0.13659824,
        0.03297533,  0.23494276], dtype=float32), array([-0.13015747, -0.0990373 , -0.10758118, ..., -0.03415059,
       -0.02957566,  0.03130121], dtype=float32)]
[[1.0000001  0.4095855  0.50060546 0.38853735 0.40622115 0.41964838]
 [0.4095855  0.9999999  0.5676453  0.4504942  0.46586353 0.31911105]
 [0.50060546 0.5676453  0.9999998  0.4529013  0.44194466 0.30684596]
 [0.38853735 0.4504942  0.4529013  0.99999964 0.3872245  0.32934457]
 [0.40622115 0.46586353 0.44194466 0.38

In [ ]:
# original wer: https://martin-thoma.com/word-error-rate-calculation/
def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]